# 6.1 Reading and Writing Data in Text Format 

Parsing functions in pandas:
- `read_csv`
- `read_fwf`
- `read_clipboard`
- `read_excel`
- `read_hdf`
- `read_html`
- `read_json`
- `read_msgpack`
- `read_pickle`
- `read_sas`
- `read_sql`
- `read_stata`
- `read_feather`

With optinal arguments: 
- Indexing: Can treat one or more columns as the returned DF, and whether to get column names from the file, the user, or not at all
- Type Inference and Data Conversion: Includes the **user-defined value conversions** and custom list of missing value markers.
- Datetime Parsing: Includes **combining capability**, including combining date and time info spread over multiple columns into a single column in the result.
- Iterating: Iterating over chunks of very large files.
- Unclean Data Issues: Skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

Some of these functions perform type inference because the column data types are not part of the data format. 

In [3]:
# read examples/ex1.csv
import pandas as pd 
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,2,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Notice the file has a header row. If the file you are working with does not have one, pass `header=None` **or** assign the names to the columns yourself by passing the `names` argument. 
- `df = pd.read_csv('examples/ex1.csv, header=None)`
- `df = pd.read_csv('examples/ex1.csv, names=['col1','col2', ...])`

You can indicate what column you would like to be the index column:
- `index_col='col6`

Furthermore you can create a hierarchical index (multiple index values) by passing a list of columns to the `index_col` argument. 

In cases where the data does not have a fixed delimiter, you can pass `sep` argument, and use a **regular expression** to choose the delimeter. `read_csv` can infer which column to be the DF's index. I*t does this by noticing that there is one fewer column name in the data u are passing. 

Pass a list of indeces to the `skiprows` argument to skip those rows when loading in the data.  

 Handling missing values is an important and frequently nuanced part of the file parsing process. Missing data is usually either not present (empty string) or marked by some **sentinel** value. By default, pandas uses a set of commonly occurring sentinels such as `NA` and `NULL`. 
 - Use `pd.isnull(data_frame)` to return a boolean DF indicating missing values. Furthermore you can use this as a mask :)
- Pass a list, dict, or a set of strings to the `na_values` argument to assign missing values.
- You can assign different `NA` sentinels to each column, just pass a dict to the `na_values` argument with the column name as the key and the sentinel as the value.


Common `read_csv` function arguments:
- `path`
- `sep` or `delimeter`
- `header` a row number to use as column names
- `index_col`
- `names`
- `skiprows`
- `na_values`
- `comment`
- `parse_dates`
- `keep_date_col`
- `converters`
- `dayfirst`
- `date_parser` a function to use to parse data
- `nrows`
- `iterator`
- `chunksize`
- `skip_footer`
- `verbose`
- `encoding`
- `squeeze` Returns a series if the parsed data only contains one column.
- `thousands`

## Reading Text Files in Pieces

Use the `nrows` argument with `pd.read_csv()` to limit the number of rows to load in. Alternatively, iterate over the file according to chunk size using `chunksize` argument 
- Create a `TextFileReader` chunk object: `chunker = pd.read_csv('path', cunksize=1000)`
- Now use this object to iterate over the file.

In [ ]:
# aggregate the counts in the 'key' column of our data. 

# total = pd.Series([])
# for piece in chunker: 
#     total.add(piece['key'].value_counts(), fill_value = 0)
# total = sort_values(ascending=False)

This code returns a series `total` with the index as the column specified `key` and the values are the value counts. 

## Writing Data to Text Format

Use the `to_csv` method. 
- `sep` can be used here for delimiter.
- By default, missing values appear as empty strings in the output. Pass `na_rep` to change this.
- Both the rown and column labels are written by default. This can be disabled with `index=False`, `header=False`.
- Write only a subset of the columns by passing a list of column names to `columns` argument
- Series also have a `to_csv` method.  

## Working with Delimited Formats 

For any file with a single-character delimiter, you can use Pythons built-in csv module. 

In [5]:
import csv 
f = open('examples/ex7.csv')
reader = csv.reader(f)

In [6]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


Now lets put the data in the form that we need.

In [10]:
with open('examples/ex7.csv') as f:
    lines=list(csv.reader(f))

# split the lines into the header line and the data lines:
header, values = lines[0], lines[1:]

# create a dict of data columns using dict comprehension and the expression zip(*values), which transpose rows to columns. 
data_dict = {h:v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

Since CSV files come in many different flavors, we can define a new format with a different delimiter, string quoting convention, or line terminator. Define a simple subclass of `csv.Dialect`

In [ ]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

reader = csv.reader(f, dialect = my_dialect)

If you dont need to go this far with it, you can simply pass one of these as an argument to `csv.reader`.

CSV Dialect Options:
- `delimiter`
- `lineterminator`
- `quotechar`
- `quoting`
- `skipinitialspace`
- `doublequote`
- `escapechar`

Note: For files with more complicated or fixed multicharacter delimiters, you will not be able to use the `csv` module. In those cases, you will have to do the line splitting and other cleanup using string's `split` method or the regular expression method `re.split`.

## JSON Data 

(JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV. JSON is nearly python code with the exception of  its null value `null` and some other nuances.

The basic types are:
- objects (dicts)
- arrays (lists)
- strings
- numbers
- booleans
- nulls

Note: **all of these keys in an object must be strings**. 

`json` is a Python library for reading and writing JSON data. 
- To convert a JSON string to Python form, use `json.loads`.
- To convert a Python object into JSON format, use `json.dumps`.
- You can pass a list of dicts to the DataFrame constructor.
- `data_frame = pd.DataFrame(dict_obj['key3'], columns=['col1', 'col2']`
- In the above example, we have a nested dict structure, so the columns define the keys that are nested within the `key3` data.


The `pandas.read_json` can automatically convert JSON datasets in specific arrangements into a Series or DatFrame. 
- The default option for this method is to **assume that each object in the JSON array is a row in the table**.

If you need to export data from pandas to JSON, one way is to use the `to_json` methods on Series and DataFrame. 


## XML and HTML: Web Scraping

Examples of libraries that support this: 
- `lxml`
- `beautifulsoup`
- `html5lib`

Pandas has a built-in function, `read_html` which uses those libraries automatically to parse tables out of HTML files as DF objects. 
- By default, it searches for and attempts to parse all tabular data contained within `<table>` tags. The result is a list of DF objects.
- `tables = pd.read_html('dir/file.html')`

In [14]:
tables = pd.read_html('fdic_failed_bank_list.html')
len(tables) 

1

In [16]:
failures=tables[0]

In [17]:
failures.head()

,Bank Name,City,State,Cert,Acquiring Institution,Closing Date,Fund Sort ascending
0,Pulaski Savings Bank,Chicago,Illinois,28611,Millennium Bank,"January 17, 2025",10548
1,The First National Bank of Lindsay,Lindsay,Oklahoma,4134,"First Bank & Trust Co., Duncan, OK","October 18, 2024",10547
2,Republic First Bank dba Republic Bank,Philadelphia,Pennsylvania,27332,"Fulton Bank, National Association","April 26, 2024",10546
3,Citizens Bank,Sac City,Iowa,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
4,Heartland Tri-State Bank,Elkhart,Kansas,25851,"Dream First Bank, N.A.","July 28, 2023",10544


In [18]:
# compute number of bank failures by year 
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps 

0   2025-01-17
1   2024-10-18
2   2024-04-26
3   2023-11-03
4   2023-07-28
5   2023-05-01
6   2023-03-12
7   2023-03-10
8   2020-10-23
9   2020-10-16
Name: Closing Date, dtype: datetime64[ns]

In [22]:
close_timestamps.dt.month.value_counts()

Closing Date
10    3
3     2
1     1
4     1
11    1
7     1
5     1
Name: count, dtype: int64

Since the object `close_timestamps` is of `dtype` `datetime64`, it has attributes such as `.dt.year`, and `.dt.month`, .etc

### Parsing XML with `lxml.objectify`

**(eXtensible Markup Language)** is another common structured data format supporting hierarchical, nested data with metadata. 
- XML and HTML are structured similarly, but XML is more general. 

Using `lxml.objectify`, we parse the file and get a reference to the root node of the XML file wth `getroot`
- `from lxml import objectify` 
- `path = 'path_to_xml'`
- `parsed = objectify.parse(open(path))`
- `root = parsed.getroot()`

`root.INDICATOR` returns a generator yielding each `<INDICATOR>` XML element. 

...

# 6.2 Binary Data Formats

**serialization** is another word for storing data. 

One of the easiest ways to store data efficiently in binary is using Pythons built-in pickle serialization. 
- Pandas objects have a `to_pickle` method the writes data to disk in pickle format

In [27]:
# load a DF from a csv file
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,2,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [28]:
# save to pickle in the same dir
frame.to_pickle('examples/frame_pickle') # writes to a new file called 'frame_pickle'

In [29]:
# read it back 
pd.read_pickle('examples/frame_pickle')

,a,b,c,2,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Note: Pickle is only recommended as a short term storage format. A file pickled today may not unpicle with a later version of a library.

Pandas also has support for two more binary data formats:
- HDF5
- Message-Pack

## Using HDF5 Format

Well-regarded file format intended for storing large qty's of scientific array data. 
- **"Hierarchical Data Format"**
- Each HDF5 file can store multiple datasets and supporting metadata.
- Supports on the fly compression with a variety of compression modes, enabling data with repeated patterns to be stored more efficiently.
- Good choice for large datasets that dont fit into memory.
- Pandas has a `HDF5Store` class that works like a dict and handles the low-level details.

In [ ]:
# import numpy as np
# frame = pd.DataFrame({'a': np.random.randn(100)})
# store = pd.HDFStore('mydata.h5')
# store['obj1'] = frame 
# store['obj1_col'] = frame['a']
# store

Objects contained in the HDF5 file cna then be retreived with the same dict-like API
- `store['obj1']`

HDF5 supports two storage schemas, `fixed` and `table`. The latter is generally slower, but it supports query operations using a special syntax. 
- `store.put('obj2', frame, format='table')` An explicit version of the `store['obj2']=frame` method but allows us to set other options like the storage format. 
- `store.select('obj2', where=['index >=10 and index <=15'])`

The `pandas.read_hdf` gives us a shortcut to these tools. 
- `frame.to_hdf('mydata.h5', 'obj3', 'format='table')`
- `pd.read_hdf('mydata.h5', 'obj3', where = ['index < 5'])` 


If you are working with large quantities of data locally, i would encourage you to explore PyTables and h5py to see how they can suit your needs. Since many data analysis problems are I/O bound rather than CPU bound, using a tool like HDF5 can massively accelerate your applications. 

### Reading Microsoft Excel Files 

(tabular data) using either the `ExcelFile` class or `pandas.read_excel` function. 
- Internally these tools use the add-on packages `xlrd` and `openpyxl` to read XLS and XLSX files.
- `xlsx_file = pd.ExcelFile('examples/ex1.xlsx')`
- `pd.read_excel(xlsx_file, 'Sheet1')`

Or you can do it in one step with: 
- `xlsx_file = pd.ExcelFile('path/to/file.xlsx', 'Sheet1')`



To write pandas data to Excel format, you first make `EscelWriter` object, then write data to it using pandas object method `to_excel`.

# 6.3 Interact With Web API's

Many websites have public api's providing data feeds via JSON or some other format. There are a number of ways to access these API's from Python. One is the `requests` package. 

In [37]:
# find the last 30 GH issues for pandas on GH 
import requests
url='https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

The response objects json method will return a dictionary containing JSON parsed into native Python Objects. 

In [41]:
data = resp.json()
data[0]['title']

'ENH: Create infrastructure for translations'

 Each element in `data` is a dict containing all of the data found on a GH issue page (except for the comments). We can pass `data` directly to DataFrame

In [44]:
issues = pd.DataFrame(data, columns=['number','title','labels','state'])
issues.head()

,number,title,labels,state
0,61220,ENH: Create infrastructure for translations,[],open
1,61219,Fix #58421: Index[timestamp[pyarrow]].union wi...,[],open
2,61218,QST: Should the absence of tzdata package affe...,"[{'id': 34444536, 'node_id': 'MDU6TGFiZWwzNDQ0...",open
3,61217,BUG: unstack incorrectly reshuffles data when ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,61216,BUG: OverflowError when fillna on DataFrame wi...,[],open


# 6.4 Interacting with Databases

Loading data from SQL into a DF is fairly straightforward. 

In [45]:
# create sqlite database 
import sqlite3
query="""
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20), c REAL, d INTEGER);"""
conn = sqlite3.connect('mydata.sqlite')
conn.execute(query)

conn.commit()

# insert a few rows 
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

statement = "INSERT INTO test VALUES(?,?,?,?)"

conn.executemany(statement, data)
conn.commit()

Most Python SQL drivers return a list of tuples when selecting data from a table

In [46]:
cursor = conn.execute("select * from test")
rows = cursor.fetchall()
rows # list of tuples 
                      

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

Naturally, you can pass the list of tuples to DataFrame constructor. We need the oclumn names as well, which we can find in the cursor's `description`. 

In [47]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [68]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


Pandas has a `read_sql` function that enables you to read data easily from a general SQLAlchemy connection. 

In [ ]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.swlite')
pd.read_sql('select * from test